In [ ]:
#! pip install pandas
#! pip install numpy 
#! pip install scikit-learn


In [62]:
import pandas as pd
import numpy as np


In [63]:
#import dataset
data = pd.read_csv('language_dataset.csv')

MINOR EDA   

In [64]:
#check for the shape of dataset and missing values
data.shape
data.isnull().sum()

(22000, 3)

In [66]:
#drop unnecessary column
data.drop(columns='Unnamed: 0', inplace=True)
data.columns

Index(['Text', 'language'], dtype='object')

In [67]:
#check for number of observations per language
data['language'].value_counts()

language
Estonian      1000
Swedish       1000
English       1000
Russian       1000
Romanian      1000
Persian       1000
Pushto        1000
Spanish       1000
Hindi         1000
Korean        1000
Chinese       1000
French        1000
Portugese     1000
Indonesian    1000
Urdu          1000
Latin         1000
Turkish       1000
Japanese      1000
Dutch         1000
Tamil         1000
Thai          1000
Arabic        1000
Name: count, dtype: int64

MULTINOMIAL LOGISTIC REGRESSION MODEL BUILDING

In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


In [75]:
#specify model type and feature extraction type
vectorizer = CountVectorizer()
multi_log = LogisticRegression(multi_class= 'multinomial')

In [ ]:
#separate X and Y
x_data = data[['Text']]
y_data = data[['language']]

In [80]:
#extract X features and check new shape
x_data_vectorized = vectorizer.fit_transform(x_data['Text'])
x_data_vectorized.shape

In [82]:
#split X and Y data with balanced number of observations in target variable
x_train, x_test, y_train, y_test = train_test_split(x_data_vectorized, y_data, test_size= 0.3, random_state=34, stratify = y_data)

In [ ]:

#fit log model and check for accuracy
multi_model = multi_log.fit(x_train, y_train['language'])
multi_model.score(x_test, y_test)

#94.6%

0.9466666666666667

LOG MODEL METRICS EVALUATION

In [131]:

from sklearn.metrics import classification_report


In [ ]:
#check full metrics report
y_predicted = multi_model.predict(x_test)
metrics_report_multi_log = classification_report(y_test, y_predicted)
print(metrics_report_multi_log)

              precision    recall  f1-score   support

      Arabic       1.00      0.97      0.99       300
     Chinese       0.80      0.51      0.63       300
       Dutch       1.00      0.98      0.99       300
     English       0.89      0.98      0.93       300
    Estonian       0.99      0.93      0.96       300
      French       1.00      1.00      1.00       300
       Hindi       1.00      0.97      0.98       300
  Indonesian       0.99      0.98      0.99       300
    Japanese       0.53      0.93      0.67       300
      Korean       1.00      0.93      0.97       300
       Latin       0.96      0.96      0.96       300
     Persian       1.00      0.97      0.98       300
   Portugese       0.98      0.98      0.98       300
      Pushto       1.00      0.96      0.98       300
    Romanian       1.00      0.98      0.99       300
     Russian       0.99      0.93      0.96       300
     Spanish       0.99      0.98      0.98       300
     Swedish       1.00    